In [1]:
import pandas as pd
import numpy as np
import PreProcessing
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scalertrain = MinMaxScaler(feature_range=(0,1))
scalertest = MinMaxScaler(feature_range=(0,1))
from keras.models import Model, Sequential
from keras.layers import Dense, Input, LSTM, Dropout
from keras import regularizers
import math
import statistics
import datetime
from pyFTS.common import Util
from pyFTS.benchmarks import Measures
from sklearn.metrics import mean_absolute_error

2021-09-28 08:49:43.632281: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-28 08:49:43.632308: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv('energydata_complete.csv')
test = PreProcessing.PreProcessing(df, 'Appliances')
test.clean(['index', 'date','rv1','rv2','T9','T6','Windspeed'])
test.shift(1,1)
data = test.data

In [3]:
data['Appliances(t)'].values.reshape(-1,1)

array([[ 60],
       [ 50],
       [ 50],
       ...,
       [270],
       [420],
       [430]])

In [4]:
def sliding_window_LSTM(data,n_windows,train_size): 
    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "smape": [],
         "mae": []
    }

    order = 1
    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
        if len(ttest) > 0:
        
            X_train = ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
            X_test = ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
            y_train = ttrain['Appliances(t)']
            y_test = ttest['Appliances(t)']
            
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.fit_transform(X_test)
            y_train = scalertrain.fit_transform(y_train.values.reshape(-1,1))
            y_test = scalertest.fit_transform(y_test.values.reshape(-1,1))
            
            X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
            X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
            y_train = y_train.reshape((y_train.shape[0], 1))
            y_test = y_test.reshape((y_test.shape[0], 1))
            
        
            print('-' * 20)
            print(f'training window {(ct)}')
            
            # design network
            from keras.callbacks import EarlyStopping
            model = Sequential()
            model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activity_regularizer=regularizers.l2(10e-5),
               activation='relu', kernel_regularizer=regularizers.l1(10e-5), recurrent_regularizer = regularizers.l2(10e-5)))
            model.add(LSTM(30, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='relu'))
            model.add(LSTM(10, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False, activation='relu'))
#model.add(LSTM(5, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=False, activation='relu'))
#model.add(LSTM(1, input_shape=(train_X.shape[1], train_X.shape[2]))) 
#model.add(Dropout(0.2))
            model.add(Dense(1, activation='sigmoid'))

            model.compile(loss='mse', optimizer='adam',metrics=['mse'])

            epochs = 75
            batch_size=10
            import matplotlib.pyplot as plt
            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.2,  shuffle=False)
            yhat = model.predict(X_test)
            
            y_test = scalertest.inverse_transform(y_test)
            forecast = scalertest.inverse_transform(yhat)
            
            
            print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
            rmse = Measures.rmse(y_test,forecast)
            mape = Measures.mape(y_test,forecast)
            smape = Measures.smape(y_test,forecast)
        
            forecast = pd.DataFrame(forecast)
            forecast.fillna(forecast.mean(),inplace=True)
            forecast = np.array(forecast).reshape(-1)
            mae = mean_absolute_error(y_test, forecast)

        
            result["rmse"].append(rmse)
            result["mape"].append(mape)
            result["smape"].append(smape)
            result["mae"].append(mae)
            result["window"].append(ct)
        
            
    measures = pd.DataFrame(result)
    return measures

In [ ]:
meas = sliding_window_LSTM(data,30,0.75)

In [6]:
measures = meas
rmse = round(statistics.mean(measures['rmse']),3)
mape = round(statistics.mean(measures['mape']),3)
smape = round(statistics.mean(measures['smape']),3)
mae = round(statistics.mean(measures['mae']),3)

In [7]:
print (rmse)
print (mape)
print (smape)
print (mae)

71.329
44.748
19.341
42.59


In [8]:
df = pd.read_csv('HomeC.csv')
data = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
data = data.loc[0: : 10]
data.dropna()

/usr/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,use [kW],gen [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
0,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
10,1.585083,0.003417,0.000050,0.022100,0.678733,0.440200,0.121450,0.007433,0.013583,0.000350,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
20,0.533817,0.003450,0.000000,0.020633,0.062967,0.270033,0.004950,0.007033,0.013117,0.000750,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
30,1.190950,0.003083,0.000217,0.387400,0.061883,0.256867,0.004933,0.101783,0.012533,0.000700,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
40,1.582083,0.003067,0.000050,0.022050,0.681717,0.242433,0.005550,0.122400,0.013550,0.000567,...,36.14,0.62,10.00,29.26,1016.91,9.18,282.0,0.0000,24.40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503860,1.621417,0.004183,0.000033,0.441950,0.645467,0.041767,0.129317,0.008067,0.012883,0.000317,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503870,0.896250,0.004150,0.000233,0.496633,0.063000,0.042317,0.123800,0.007850,0.012433,0.000467,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503880,1.311617,0.004167,0.000017,0.359167,0.645433,0.041917,0.005133,0.008167,0.013083,0.000650,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503890,2.059367,0.003383,0.000033,0.153367,0.643300,0.042050,0.005117,0.008283,0.013267,0.000433,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51


In [9]:
def sliding_window_LSTM(data,n_windows,train_size): 
    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "smape": [],
         "mae": []
    }

    order = 1
    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
        if len(ttest) > 0:
        
            X_train = ttrain.loc[:,'gen [kW]':'precipProbability']
            X_test = ttest.loc[:,'gen [kW]':'precipProbability']
            y_train = ttrain['use [kW]']
            y_test = ttest['use [kW]']
            
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.fit_transform(X_test)
            y_train = scalertrain.fit_transform(y_train.values.reshape(-1,1))
            y_test = scalertest.fit_transform(y_test.values.reshape(-1,1))
            
            X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
            X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
            y_train = y_train.reshape((y_train.shape[0], 1))
            y_test = y_test.reshape((y_test.shape[0], 1))
            
        
            print('-' * 20)
            print(f'training window {(ct)}')
            
            # design network
            from keras.callbacks import EarlyStopping
            model = Sequential()
            model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activity_regularizer=regularizers.l2(10e-5),
               activation='relu', kernel_regularizer=regularizers.l1(10e-5), recurrent_regularizer = regularizers.l2(10e-5)))
            model.add(LSTM(30, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='relu'))
            model.add(LSTM(10, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False, activation='relu'))
#model.add(LSTM(5, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=False, activation='relu'))
#model.add(LSTM(1, input_shape=(train_X.shape[1], train_X.shape[2]))) 
#model.add(Dropout(0.2))
            model.add(Dense(1, activation='sigmoid'))

            model.compile(loss='mse', optimizer='adam',metrics=['mse'])

            epochs = 70
            batch_size=10
            import matplotlib.pyplot as plt
            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.2,  shuffle=False)
            yhat = model.predict(X_test)
            
            y_test = scalertest.inverse_transform(y_test)
            forecast = scalertest.inverse_transform(yhat)
            
            
            print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
            rmse = Measures.rmse(y_test,forecast)
            mape = Measures.mape(y_test,forecast)
            smape = Measures.smape(y_test,forecast)
        
            forecast = pd.DataFrame(forecast)
            forecast.fillna(forecast.mean(),inplace=True)
            forecast = np.array(forecast).reshape(-1)
            mae = mean_absolute_error(y_test, forecast)

        
            result["rmse"].append(rmse)
            result["mape"].append(mape)
            result["smape"].append(smape)
            result["mae"].append(mae)
            result["window"].append(ct)
        
            
    measures = pd.DataFrame(result)
    return measures

In [10]:
meas = sliding_window_LSTM(data,30,0.75)
measures = meas
rmse = round(statistics.mean(measures['rmse']),3)
mape = round(statistics.mean(measures['mape']),3)
smape = round(statistics.mean(measures['smape']),3)
mae = round(statistics.mean(measures['mae']),3)

101/101 [==============================] - 0s 5ms/step - loss: 0.0142 - mse: 0.0101 - val_loss: 0.0131 - val_mse: 0.0090
Epoch 8/70
101/101 [==============================] - 0s 5ms/step - loss: 0.0136 - mse: 0.0097 - val_loss: 0.0128 - val_mse: 0.0089
Epoch 9/70
101/101 [==============================] - 0s 5ms/step - loss: 0.0131 - mse: 0.0093 - val_loss: 0.0124 - val_mse: 0.0087
Epoch 10/70
101/101 [==============================] - 0s 5ms/step - loss: 0.0126 - mse: 0.0090 - val_loss: 0.0122 - val_mse: 0.0085
Epoch 11/70
101/101 [==============================] - 1s 7ms/step - loss: 0.0122 - mse: 0.0087 - val_loss: 0.0119 - val_mse: 0.0083
Epoch 12/70
101/101 [==============================] - 1s 6ms/step - loss: 0.0119 - mse: 0.0084 - val_loss: 0.0116 - val_mse: 0.0081
Epoch 13/70
101/101 [==============================] - 1s 7ms/step - loss: 0.0116 - mse: 0.0082 - val_loss: 0.0113 - val_mse: 0.0079
Epoch 14/70
101/101 [==============================] - 0s 5ms/step - loss: 0.0113 -

In [11]:
print (rmse)
print (mape)
print (smape)
print (mae)

0.594
129.681
24.387
0.388


In [12]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [13]:
data = pd.read_csv('household_power_consumption.csv', sep = ";")
data = data.drop(labels=['Time','Date'], axis=1)
data = data.loc[0: : 30] # 30 minutes
data.dropna(inplace = True)
data = clean_dataset(data)
data.head()

/usr/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,4.216,0.418,234.84,18.4,0.0,1.0,17.0
30,2.720,0.000,235.06,11.6,0.0,0.0,17.0
60,3.452,0.000,235.20,15.2,0.0,1.0,17.0
90,4.298,0.000,232.39,18.4,0.0,1.0,16.0
120,3.262,0.052,232.64,14.0,0.0,0.0,17.0


In [14]:
def sliding_window_LSTM(data,n_windows,train_size): 
    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "smape": [],
         "mae": []
    }

    order = 1
    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
        if len(ttest) > 0:
        
            X_train = ttrain.loc[:,'Global_reactive_power':'Sub_metering_3']
            X_test = ttest.loc[:,'Global_reactive_power':'Sub_metering_3']
            y_train = ttrain['Global_active_power']
            y_test = ttest['Global_active_power']
            
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.fit_transform(X_test)
            y_train = scalertrain.fit_transform(y_train.values.reshape(-1,1))
            y_test = scalertest.fit_transform(y_test.values.reshape(-1,1))
            
            X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
            X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
            y_train = y_train.reshape((y_train.shape[0], 1))
            y_test = y_test.reshape((y_test.shape[0], 1))
            
        
            print('-' * 20)
            print(f'training window {(ct)}')
            
            # design network
            from keras.callbacks import EarlyStopping
            model = Sequential()
            model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activity_regularizer=regularizers.l2(10e-5),
               activation='relu', kernel_regularizer=regularizers.l1(10e-5), recurrent_regularizer = regularizers.l2(10e-5)))
            model.add(LSTM(30, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation='relu'))
            model.add(LSTM(10, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False, activation='relu'))
#model.add(LSTM(5, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=False, activation='relu'))
#model.add(LSTM(1, input_shape=(train_X.shape[1], train_X.shape[2]))) 
#model.add(Dropout(0.2))
            model.add(Dense(1, activation='sigmoid'))

            model.compile(loss='mse', optimizer='adam',metrics=['mse'])

            epochs = 50
            batch_size=10
            import matplotlib.pyplot as plt
            history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_split=0.2,  shuffle=False)
            yhat = model.predict(X_test)
            
            y_test = scalertest.inverse_transform(y_test)
            forecast = scalertest.inverse_transform(yhat)
            
            
            print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
            rmse = Measures.rmse(y_test,forecast)
            mape = Measures.mape(y_test,forecast)
            smape = Measures.smape(y_test,forecast)
        
            forecast = pd.DataFrame(forecast)
            forecast.fillna(forecast.mean(),inplace=True)
            forecast = np.array(forecast).reshape(-1)
            mae = mean_absolute_error(y_test, forecast)

        
            result["rmse"].append(rmse)
            result["mape"].append(mape)
            result["smape"].append(smape)
            result["mae"].append(mae)
            result["window"].append(ct)
        
            
    measures = pd.DataFrame(result)
    return measures

In [15]:
meas = sliding_window_LSTM(data,30,0.75)
measures = meas
rmse = round(statistics.mean(measures['rmse']),3)
mape = round(statistics.mean(measures['mape']),3)
smape = round(statistics.mean(measures['smape']),3)
mae = round(statistics.mean(measures['mae']),3)

137/137 [==============================] - 0s 3ms/step - loss: 8.2843e-04 - mse: 2.6373e-04 - val_loss: 6.8980e-04 - val_mse: 1.5229e-04
Epoch 44/50
137/137 [==============================] - 0s 3ms/step - loss: 7.9317e-04 - mse: 2.4288e-04 - val_loss: 6.6760e-04 - val_mse: 1.4377e-04
Epoch 45/50
137/137 [==============================] - 0s 3ms/step - loss: 7.6081e-04 - mse: 2.2526e-04 - val_loss: 6.4663e-04 - val_mse: 1.3610e-04
Epoch 46/50
137/137 [==============================] - 0s 3ms/step - loss: 7.3170e-04 - mse: 2.1096e-04 - val_loss: 6.2784e-04 - val_mse: 1.3035e-04
Epoch 47/50
137/137 [==============================] - 0s 3ms/step - loss: 7.0452e-04 - mse: 1.9772e-04 - val_loss: 6.1256e-04 - val_mse: 1.2685e-04
Epoch 48/50
137/137 [==============================] - 1s 4ms/step - loss: 6.7933e-04 - mse: 1.8549e-04 - val_loss: 6.0004e-04 - val_mse: 1.2418e-04
Epoch 49/50
137/137 [==============================] - 0s 4ms/step - loss: 6.6195e-04 - mse: 1.7978e-04 - val_loss: 5.

In [16]:
print (rmse)
print (mape)
print (smape)
print (mae)

0.122
17.49
7.359
0.088
